# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
cities = pd.read_csv("../WeatherPy/cities.csv")
cities.head()

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,atuona,-9.8000,-139.0333,76.50,63,44,11.34,PF,1661210391
1,bengkulu,-3.8004,102.2655,74.71,86,97,3.36,ID,1661210391
2,rikitea,-23.1203,-134.9692,70.48,70,24,14.09,PF,1661210392
3,flin flon,54.7682,-101.8650,69.64,49,40,4.61,CA,1661210392
4,bethel,41.3712,-73.4140,76.19,85,85,3.94,US,1661210393


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [24]:
# Store latitude and longitude in locations
locations = cities[["Latitude", "Longitude"]]

humidity = cities["Humidity (%)"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=4)
# Add layer
fig.add_layer(heat_layer)

# Display
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
# Ideal Temp
cities_ideal = cities.loc[(cities["Temperature (F)"] >= 65) & (cities["Temperature (F)"] <= 75)]

# Ideal Wind Speed
cities_ideal = cities_ideal.loc[cities_ideal["Wind Speed (mph)"] < 20]


# Cloudiness
hotel_df = cities_ideal.loc[cities_ideal["Cloudiness (%)"] < 10]

hotel_df.head()


,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
32,abu samrah,35.3029,37.1841,74.26,74,0,11.95,SY,1661210404
87,arraial do cabo,-22.9661,-42.0278,66.16,82,0,4.61,BR,1661210430
98,mareeba,-17.0000,145.4333,73.49,75,4,2.57,AU,1661210434
158,sault sainte marie,46.5168,-84.3333,72.16,57,0,11.50,CA,1661210178
173,krasnoarmeyskiy,47.0114,42.2072,72.03,40,0,12.19,RU,1661210469


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotel_df["Hotel Name"] = ""

C:\Users\Chris\AppData\Local\Temp/ipykernel_28104/382487186.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


In [18]:
# find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    lat = row['Latitude']
    lon = row["Longitude"]
    
    # add keyword to params dict
    params['location'] = f'{lat},{lon}'

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 32: abu samrah.
Missing field/result... skipping.
------------
Retrieving Results for Index 87: arraial do cabo.
Closest hotel is Pousada Porto Praia.
------------
Retrieving Results for Index 98: mareeba.


C:\Users\Chris\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Closest hotel is Trinity Plains Tourist Park.
------------
Retrieving Results for Index 158: sault sainte marie.
Closest hotel is Comfort Inn.
------------
Retrieving Results for Index 173: krasnoarmeyskiy.
Missing field/result... skipping.
------------
Retrieving Results for Index 185: port hedland.
Closest hotel is The Esplanade Hotel.
------------
Retrieving Results for Index 207: morondava.
Closest hotel is La Case Bambou.
------------
Retrieving Results for Index 232: ojinaga.
Closest hotel is Suites Las Palmas.
------------
Retrieving Results for Index 235: lyubar.
Closest hotel is Садиба" Над Случчю".
------------
Retrieving Results for Index 268: dingzhou.
Closest hotel is 宜家时尚宾馆.
------------
Retrieving Results for Index 320: alekseyevka.
Closest hotel is Apartment on Ploshchad Pobedy.
------------
Retrieving Results for Index 323: mount isa.
Closest hotel is ibis Styles Mt Isa Verona.
------------
Retrieving Results for Index 343: sarandi.
Closest hotel is HOTEL GRALHA AZUL (

In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [26]:
# Add marker layer ontop of heat map

marker_locations = hotel_df[['Latitude', 'Longitude']]

markers = gmaps.marker_layer(marker_locations,
    info_box_content=hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))